In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 15.0 MB/s eta 0:00:00


In [2]:
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.7 MB/s eta 0:00:00


In [3]:
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
import math
from ultralytics import YOLO  # Import YOLOv8

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
def calculate_distance(coord1, coord2):
    """Calculate Euclidean distance between two points."""
    return math.sqrt((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2)

In [5]:

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second (to calculate speed)
    tracker = DeepSort(max_age=5)  # Initialize tracker
    model = YOLO('yolov8n.pt')  # Load YOLOv8 model

    players_data = {}  # Dictionary to store player and ball data
    prev_positions = {}  # Store previous positions to calculate distance

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection using YOLOv8
        results = model(frame)  # Detect objects on the frame
        detections = []
        
        for result in results[0].boxes:
            # YOLOv8 outputs: x_min, y_min, x_max, y_max, confidence, class
            x_min, y_min, x_max, y_max = result.xyxy[0]
            conf = result.conf[0]  # Confidence score
            cls = result.cls[0]  # Class ID

            if cls in [0, 32]:  # Track 'person' (0) and 'sports ball' (32)
                bbox_xywh = [(x_min + x_max) / 2, (y_min + y_max) / 2, x_max - x_min, y_max - y_min]  # Convert to center x, center y, width, height
                detections.append((bbox_xywh, conf, cls))  # Append class ID to detections

        # Update tracks with detections
        if detections:
            tracks = tracker.update_tracks(detections, frame=frame)  # Update tracker with the current frame

            # Store data for players and sports balls
            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltwh(orig=True)  # Get bounding box (left, top, width, height)
                center = (bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2)  # Calculate center of the box

                # If we have a previous position, calculate distance traveled
                if track_id in prev_positions:
                    distance_traveled = calculate_distance(prev_positions[track_id], center)
                    players_data[track_id]['distance'] += distance_traveled
                    players_data[track_id]['speed'] = distance_traveled * fps  # Speed = distance * fps
                else:
                    players_data[track_id] = {'distance': 0, 'speed': 0}  # Initialize data for new track

                # Update previous position with the current center
                prev_positions[track_id] = center

                # Track the class for distinguishing between players and balls
                if cls == 0:  # For persons
                    players_data[track_id]['type'] = 'person'
                elif cls == 32:  # For sports balls
                    players_data[track_id]['type'] = 'ball'

    cap.release()
    return players_data

In [6]:

# Example usage
video_path = '/kaggle/input/videowe/video.mp4'  # Input video path
players_data = process_video(video_path)

# Print player and ball data
print("Player and Ball Data:", players_data)


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.1MB/s]



0: 384x640 3 persons, 157.1ms
Speed: 4.7ms preprocess, 157.1ms inference, 13.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 98.7ms
Speed: 3.1ms preprocess, 98.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 101.4ms
Speed: 4.2ms preprocess, 101.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 104.2ms
Speed: 3.1ms preprocess, 104.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 105.5ms
Speed: 5.0ms preprocess, 105.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 104.5ms
Speed: 3.4ms preprocess, 104.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 101.5ms
Speed: 4.6ms preprocess, 101.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 177.3ms
Speed: 33.9ms preprocess, 177.3ms inference, 1.2ms postprocess per i